Code to connect to a SQL server database, create tables and insert records

In [1]:
# Import libraries of interest 
import pandas as pd
import numpy as np 
import mysql.connector

In [2]:
# Enter your password...
password = ...

In [7]:
# Connect to SQL database on your computer 
conn = mysql.connector.connect(user='root', password=password, host='127.0.0.1', database='cville_climbs')
cursor = conn.cursor() # Enable access to mySQL

SQL to set up datatables, keys and constraints

In [77]:
# Create location table 
create_location = '''
CREATE TABLE location 
(location_id INT NOT NULL AUTO_INCREMENT,
 latitude DECIMAL NOT NULL,
 longitude DECIMAL NOT NULL, 
 name VARCHAR(32) NOT NULL,
 entrance_city VARCHAR(255),
 entrance_zip VARCHAR(255),
 entrance_address VARCHAR(255),
 num_routes INT NOT NULL,
 PRIMARY KEY (location_id))
'''
cursor.execute(create_location)
conn.commit()


# Create person table 
create_person = '''
CREATE TABLE person
(person_id INT NOT NULL AUTO_INCREMENT,
 f_name VARCHAR(32) NOT NULL,
 l_name VARCHAR(32) NOT NULL,
 email VARCHAR(255) NOT NULL,
 phone INT NOT NULL,
 type_person enum('RC','C','A') NOT NULL,
 PRIMARY KEY (person_id))
'''
cursor.execute(create_person)
conn.commit()


# Create route table
create_route = '''
CREATE TABLE route 
(route_id INT NOT NULL AUTO_INCREMENT, 
 name VARCHAR(32) NOT NULL, 
 difficulty DECIMAL NOT NULL,
 rating DECIMAL,
 type enum('indoor','outdoor') NOT NULL,
 person_id INT NOT NULL,
 location_id INT NOT NULL,
 PRIMARY KEY (route_id),
 FOREIGN KEY (person_id) REFERENCES person (person_id),
 FOREIGN KEY (location_id) REFERENCES location (location_id))
'''
cursor.execute(create_route)
conn.commit()


# Create log table 
create_log = '''
CREATE TABLE log 
(log_id INT NOT NULL AUTO_INCREMENT, 
 person_id INT NOT NULL, 
 comment LONGTEXT NOT NULL,
 route_beta LONGTEXT,
 route_id INT NOT NULL,
 PRIMARY KEY (log_id),
 FOREIGN KEY (route_id) REFERENCES route(route_id),
 FOREIGN KEY (person_id) REFERENCES person(person_id))
'''
cursor.execute(create_log)
conn.commit()


# Create route_setting table 
create_route_setting = '''
CREATE TABLE route_setting
(route_id INT NOT NULL,
 person_id INT NOT NULL,
 PRIMARY KEY (route_id, person_id),
 FOREIGN KEY (route_id) REFERENCES route(route_id),
 FOREIGN KEY (person_id) REFERENCES person(person_id))
'''
cursor.execute(create_route_setting)
conn.commit()


# Create route setter table
create_route_setter = '''
CREATE TABLE route_setter
(person_id INT NOT NULL,
 n_routes_created INT NOT NULL,
 route_setting_exp_lvl DECIMAL NOT NULL,
 certifications LONGTEXT,
 PRIMARY KEY (person_id),
 FOREIGN KEY (person_id) REFERENCES person (person_id))
'''
cursor.execute(create_route_setter)
conn.commit()


# Create climber table 
create_climber = '''
CREATE TABLE climber
(person_id INT NOT NULL,
 exp_lvl DECIMAL NOT NULL, 
 routes_attempted INT NOT NULL,
 is_team_member BOOLEAN NOT NULL,
 PRIMARY KEY (person_id),
 FOREIGN KEY (person_id) REFERENCES person (person_id))
'''
cursor.execute(create_climber)
conn.commit()


# Create administrator table 
create_admin = '''
CREATE TABLE admin
(person_id INT NOT NULL,
 has_edit_permission BOOLEAN NOT NULL, 
 has_delete_permission BOOLEAN NOT NULL, 
 has_create_permission BOOLEAN NOT NULL, 
 is_owner BOOLEAN NOT NULL,
 PRIMARY KEY (person_id),
 FOREIGN KEY (person_id) REFERENCES person (person_id))
'''
cursor.execute(create_admin)
conn.commit()

In [76]:
# Remove all tables from database 
destroy_tables = '''
DROP TABLE IF EXISTS 
route, location, log, route_setting, person, route_setter, admin, climber 
'''
cursor.execute(destroy_tables)
conn.commit()

In [ ]:
# Close your connection 
conn.close()